In [85]:
import pandas as pd
import os
import platform


os_type = platform.system()
os_file_path =os.getcwd()
if os_type == "Darwin": #running on mac
    relative_file_path = "/Data/retail_sales.csv"
else: #running on windows or linux
    relative_file_path = "\\Data\\retail_sales.csv"

data_file_path = os_file_path + relative_file_path
print(data_file_path)

df = pd.read_csv(data_file_path)
df.head()

df.info()


/Users/mark/Documents/IRO/HCIIV/Project repo/Data/retail_sales.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Store ID               30000 non-null  object 
 1   Product ID             30000 non-null  int64  
 2   Date                   30000 non-null  object 
 3   Units Sold             30000 non-null  int64  
 4   Sales Revenue (USD)    30000 non-null  float64
 5   Discount Percentage    30000 non-null  int64  
 6   Marketing Spend (USD)  30000 non-null  int64  
 7   Store Location         30000 non-null  object 
 8   Product Category       30000 non-null  object 
 9   Day of the Week        30000 non-null  object 
 10  Holiday Effect         30000 non-null  bool   
dtypes: bool(1), float64(1), int64(4), object(5)
memory usage: 2.3+ MB


Data exploration

In [86]:
#print(df.info())
print(f"Mean of sales revenue: {df['Sales Revenue (USD)'].mean()}")
print(f"Median of sales revenue: {df['Sales Revenue (USD)'].median()}")
print(f"Mean of discount: {df['Discount Percentage'].mean()}")
print(f"Mean of marketing spending: {df['Marketing Spend (USD)'].mean()}")
print(f"Product categories: {df['Product Category'].value_counts()}")


Mean of sales revenue: 2749.509592666667
Median of sales revenue: 1902.42
Mean of discount: 2.9738333333333333
Mean of marketing spending: 49.94403333333333
Product categories: Product Category
Furniture      9503
Electronics    8041
Clothing       6608
Groceries      5848
Name: count, dtype: int64


In [87]:
#creating new columns

#creatign the discounted price

#filtering out cases with no units sold, cause there we have no idea of price
atLeastOneSold = df['Units Sold'] >= 1
df = df[atLeastOneSold]

def discounted_price(row):
    units_sold = row['Units Sold']
    revenue = row['Sales Revenue (USD)']
    return round(revenue/units_sold, 2)

df['Discounted Price'] = df.apply(discounted_price, axis=1)
df.head()


#getting the full price

def full_price(row):
    discount_percentage = row['Discount Percentage']
    discounted_price = row['Discounted Price']
    if discount_percentage == 0:
        return discounted_price
    else:
        price_percentage = (100 - discount_percentage) / 100
        return round(discounted_price / price_percentage, 2)

df['Full Price'] = df.apply(full_price, axis=1)
df.head()

,Store ID,Product ID,Date,Units Sold,Sales Revenue (USD),Discount Percentage,Marketing Spend (USD),Store Location,Product Category,Day of the Week,Holiday Effect,Discounted Price,Full Price
0,Spearsland,52372247,2022-01-01,9,2741.69,20,81,Tanzania,Furniture,Saturday,False,304.63,380.79
1,Spearsland,52372247,2022-01-02,7,2665.53,0,0,Mauritania,Furniture,Sunday,False,380.79,380.79
2,Spearsland,52372247,2022-01-03,1,380.79,0,0,Saint Pierre and Miquelon,Furniture,Monday,False,380.79,380.79
3,Spearsland,52372247,2022-01-04,4,1523.16,0,0,Australia,Furniture,Tuesday,False,380.79,380.79
4,Spearsland,52372247,2022-01-05,2,761.58,0,0,Swaziland,Furniture,Wednesday,False,380.79,380.79


Model specification:

Predicted value: Sales Revenue
Predictor values:
    Discount percentage
    Marketing spending
    Product category
    Day of the week
    Holiday effect
    Full price

Omited variables from the model:
    Store id
    product ID
    date
    Units sold
    store location
    Discounted price

Discounted price and units sold are omited since they would be problematic, and cause data leakage

In [101]:
df_model = df[['Sales Revenue (USD)', 'Discount Percentage', 'Marketing Spend (USD)', 'Product Category', 'Day of the Week', 'Holiday Effect', 'Full Price']]


#creating dummy variables for product category and day of the week
df_model = pd.get_dummies(df_model, columns = ['Product Category'])
df_model = pd.get_dummies(df_model, columns = ['Day of the Week'])

#dropping a reference dummy for each
df_model.drop('Product Category_Clothing',
  axis='columns', inplace=True)

df_model.drop('Day of the Week_Monday',
  axis='columns', inplace=True)

#dividing into train and test, stratifying by product category

from sklearn.model_selection import train_test_split

train, test = train_test_split(df_model, test_size=0.3, stratify=df_model[['Product Category_Electronics', 'Product Category_Furniture', 'Product Category_Groceries']])


print(f'Train shape: {train.shape}, test shape: {test.shape}')

#spitting into X and Y



X_train = train.drop('Sales Revenue (USD)', axis='columns', inplace=False)
y_train = train['Sales Revenue (USD)']
X_test = test.drop('Sales Revenue (USD)', axis='columns', inplace=False)
y_test = test['Sales Revenue (USD)']





Train shape: (20834, 14), test shape: (8929, 14)


In [108]:
#Running 3 types of regression models to get an idea of which one works best

from sklearn import metrics
from time import perf_counter
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

m1=HistGradientBoostingRegressor()
t_start = perf_counter()
m1.fit(X_train, y_train)
t_stop = perf_counter()
y_predict = m1.predict(X_test)
R2 = metrics.r2_score(y_test, y_predict)
print(f"Hist gradient model R2 is {round(R2, 3)}.")
print(f"Time taken: {t_stop - t_start}")

#Testing RandomForrestClassifier


m2=RandomForestRegressor()
t_start = perf_counter()
m2.fit(X_train, y_train)
t_stop = perf_counter()
y_predict = m2.predict(X_test)
accuracy = metrics.r2_score(y_test, y_predict)
print(f"Random forrest classifier R2 is {round(R2, 3)}.")
print(f"Time taken: {t_stop - t_start}")



m3=GradientBoostingRegressor()
t_start = perf_counter()
m3.fit(X_train, y_train)
t_stop = perf_counter()
y_predict = m3.predict(X_test)
accuracy = metrics.r2_score(y_test, y_predict)
print(f"Gradient boosting classifier R2 is {round(R2, 3)}.")
print(f"Time taken: {t_stop - t_start}")

#The 3 models perform exactly the same
#Quickest model is hist gradient, so going with this one

Hist gradient model R2 is 0.559.
Time taken: 0.36288898199927644
Random forrest classifier R2 is 0.559.
Time taken: 3.9214383489998
Gradient boosting classifier R2 is 0.559.
Time taken: 0.991855188000045


In [111]:
#training the model

from sklearn.model_selection import GridSearchCV
from joblib import dump, load

m4 = HistGradientBoostingRegressor()

#Hyperparameter optimisation
param_options = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_iter': [100, 200, 500],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [20, 50, 100],
    'max_leaf_nodes': [31, 63, 127],
    'l2_regularization': [0.0, 0.1, 1.0],
    'loss': ['squared_error', 'absolute_error'],
    'max_bins': [255, 512],
}

clf = GridSearchCV(estimator = m4,
                   param_grid = param_options,
                   cv = 4)

clf.fit(X_train, y_train)
print(f"Best hyperparameter combination is: {clf.best_params_}")
best_model = clf.best_estimator_
dump(best_model, 'best_model.joblib')